In [70]:
from dotenv import load_dotenv
from pydantic import BaseModel, Field
from typing import List, Literal
import os

In [71]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_groq import ChatGroq
from langchain.agents.middleware import SummarizationMiddleware 
from langgraph.checkpoint.memory import InMemorySaver
from langchain.agents import create_agent

In [72]:
load_dotenv()
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")
os.environ["TAVILY_API_KEY"]=os.getenv("TAVILY_API_KEY")

In [73]:
class ResearchReport(BaseModel):
    title: str = Field(description="Catchy title")
    summary: str = Field(description="2-sentence summary")
    sources: List[str] = Field(description="List of URLs")
    confidence_score: float = Field(description="0 to 1")

class ValidationResult(BaseModel):
    is_valid: bool = Field(description="True if facts are dated for 2026")
    critique: str = Field(description="Explanation of gaps")
    # Using a literal ensures the LLM picks ONLY one of these two strings
    needed_action: Literal["finalize", "research_more"] = Field(
        description="Must be 'finalize' or 'research_more'"
    )


In [74]:



llm = ChatGroq(model="qwen/qwen3-32b", temperature=0) # Use 0 temp for extraction
tools = [TavilySearchResults(max_results=3)]
memory = InMemorySaver()

In [75]:
Validaiton_PROMPT = """You are an expert Research Agent. 
Step 1: Use your tools to find information.
Step 2: Self-Reflect. Look at your findings. If they are vague or missing dates, search again.
Step 3: Only provide the final answer once you are confident the data is accurate for 2026."""

Resarch_PROMPT = """You are a professional researcher. Search the web and then provide a structured report."""


In [90]:
# sumMiddleware=SummarizationMiddleware(
#     model=llm,
#     trigger=("messages",2),
#     keep=("messages",10)
# )


sum_middleware = SummarizationMiddleware(
    model=llm,
    trigger=("tokens", 500), # Summarize after 5 messages
    keep=("tokens", 10000)      # Keep the most recent 2
)

In [77]:


research_parser = llm.with_structured_output(ResearchReport)
validation_parser = llm.with_structured_output(ValidationResult)

In [78]:

research_agent = create_agent(
    model=llm,
    tools=tools,
    checkpointer=memory,
    middleware=[sum_middleware],
    system_prompt="You are a 2026 research specialist. Search and find hard facts."
)

In [79]:
def run_validated_research(query: str):
    config = {"configurable": {"thread_id": "starship_2026"}}
    
    # Step 1: Research
    print("Researching...")
    state = research_agent.invoke({"messages": [("user", query)]}, config=config)
    raw_draft = state["messages"][-1].content
    
    # Step 2: Validate (Using the parser to enforce the schema)
    print("Validating...")
    # Instruction to follow the schema strictly
    v_prompt = f"Critique this research for 2026 accuracy. Output must include needed_action.\n\n{raw_draft}"
    
    try:
        validation = validation_parser.invoke(v_prompt)
    except Exception:
        # Emergency fallback 
        validation = ValidationResult(is_valid=False, critique="Schema failed", needed_action="research_more")

    if not validation.is_valid:
        print(f"Validation Failed: {validation.critique}")
        # Step 3: Revise
        state = research_agent.invoke(
            {"messages": [("user", f"Fix your report based on this: {validation.critique}")]},
            config=config
        )
        raw_draft = state["messages"][-1].content

    # Step 4: Final Structure
    return research_parser.invoke(raw_draft)

In [80]:
report = run_validated_research("What are the latest updates on SpaceX Starship 2026?")


Researching...
Validating...
Validation Failed: The research contains speculative claims about 2026 events (e.g., March 2026 launch, V3 details) that cannot be verified as of October 2023. Specifics like 'six weeks from January 26, 2026' and 'structural anomaly in late 2025' are future events with no current documentation. Sources like Wikipedia and the cited articles (from 2026) do not exist in the present timeline. The timeline assumes rapid Starship iteration (12th flight test by 2026) inconsistent with SpaceX’s historical delays.


In [81]:
report

ResearchReport(title='SpaceX Starship Development Update: Key Milestones and Future Plans (October 2023)', summary='As of October 2023, SpaceX has completed five Starship flight tests, with the latest achieving a successful high-altitude flight and booster landing. Upcoming plans focus on 2024 test flights and Raptor V3 engine integration by 2025, pending regulatory approvals.', sources=['https://en.wikipedia.org/wiki/Starship_(spacecraft)', 'https://www.faa.gov/', 'https://www.spacex.com/', 'https://www.nasa.gov/artemis'], confidence_score=0.9)

In [82]:
print("\n--- FINAL STRUCTURED REPORT ---")
print(f"Title: {report.title}")
print(f"Summary: {report.summary}")
print(f"Sources: {report.sources}")
print(f"Confidence: {report.confidence_score}")


--- FINAL STRUCTURED REPORT ---
Title: SpaceX Starship Development Update: Key Milestones and Future Plans (October 2023)
Summary: As of October 2023, SpaceX has completed five Starship flight tests, with the latest achieving a successful high-altitude flight and booster landing. Upcoming plans focus on 2024 test flights and Raptor V3 engine integration by 2025, pending regulatory approvals.
Sources: ['https://en.wikipedia.org/wiki/Starship_(spacecraft)', 'https://www.faa.gov/', 'https://www.spacex.com/', 'https://www.nasa.gov/artemis']
Confidence: 0.9


In [ ]:
config = {"configurable": {"thread_id": "test_session"}}

# 1. Force a long conversation to hit the trigger (e.g., 5 messages)
print("--- Sending messages to trigger summarization ---")
for i in range(3):
    research_agent.invoke({"messages": [("user", f"Tell me fact number {i} about Starship")]}, config=config)

# 2. Get the state properly
state = research_agent.get_state(config)

# 3. Access the messages
messages = state.values.get("messages", [])

print(f"Total messages in history: {len(messages)}")

# 4. Verification Logic
if len(messages) < 6:
    print("Summarization successful! The message list was truncated.")
    print(f"Earliest message now is: {messages[0].content}...")
else:
    print("Summarization not triggered. Check your middleware 'trigger' count.")

--- Sending messages to trigger summarization ---
Total messages in history: 4
Summarization successful! The message list was truncated.
Earliest message now is: Here is a summary of the conversation to date:

<t...


In [96]:
state.values["messages"]

[HumanMessage(content='Here is a summary of the conversation to date:\n\n<think>\nOkay, let\'s see. The user is asking for fact number 1 about Starship. From the conversation history, I need to extract the most relevant context.\n\nLooking back, the user previously asked for fact 0, and the assistant provided details about Starship\'s structure and reusability. Now, the user is moving on to fact 1. In the history, there\'s a mention of the Raptor engines and propulsion system as the next fact. The assistant had outlined that fact 1 would cover the Raptor engines, their design, and the use of methane-based fuel.\n\nI need to structure this into the required sections. The session intent is still the user\'s goal to get a series of facts about Starship. The summary should include the key points about the Raptor engines, their specifications, and why methane is used. Artifacts are none since no files are mentioned. Next steps would be to present fact 1 and ask if the user wants to proceed 